In [ ]:
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
%matplotlib inline


import xgboost as xgb
from sklearn import model_selection, preprocessing


pd.set_option('display.max_columns', 500)

In [ ]:
train_df = pd.read_csv('../input/train.csv',parse_dates=['timestamp'])
result_df = pd.read_csv('../input/test.csv',parse_dates=['timestamp'])
train_df.shape
combine=[train_df,result_df]

In [ ]:
for c in train_df.columns:
    if train_df[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[c].values)) 
        train_df[c] = lbl.transform(list(train_df[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in result_df.columns:
    if result_df[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(result_df[c].values)) 
        result_df[c] = lbl.transform(list(result_df[c].values))
        #x_test.drop(c,axis=1,inplace=True)        

In [ ]:
train_df = train_df.sample(n=30471,replace=True)

In [ ]:
X_train = train_df[(train_df.price_doc != 1000000)&
                   (train_df.price_doc != 2000000)
                  ].drop(['price_doc','id','timestamp'],axis=1)
Y_train = train_df[(train_df.price_doc != 1000000)&
                   (train_df.price_doc != 2000000)
                  ]['price_doc'].values.reshape(-1,1)
X_train.shape

In [ ]:
X_result = result_df.drop(['id','timestamp'],axis=1)
id_test = result_df['id']
X_result.shape

In [ ]:
dtrain = xgb.DMatrix(X_train, Y_train)
dresult = xgb.DMatrix(X_result)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
# Uncomment to tune XGB `num_boost_rounds`
#model = xgb.cv(xgb_params, dtrain, num_boost_round=1000,
                  #early_stopping_rounds=20, verbose_eval=10)
model = xgb.train(xgb_params, dtrain, num_boost_round=400, verbose_eval=50, 
                  evals=[(dtrain,'val')])

In [ ]:
y_pred=model.predict(dresult)
output=pd.DataFrame(data={'price_doc':y_pred},index=id_test)

In [ ]:
first_pred = y_pred

In [ ]:
second_pred = y_pred
second_pred

In [ ]:
third_pred = y_pred

In [ ]:
output=pd.DataFrame(data={'price_doc':(first_pred+second_pred+third_pred)/3},index=id_test)
output.head(10)

In [ ]:
output.to_csv('output.csv',header=True)

In [ ]:
pd.DataFrame(data={'price1':first_pred,'price2':second_pred,'price3':third_pred,
                  'price12':(first_pred+second_pred)/2,
                  'price13':(first_pred+third_pred)/2,
                  'price23':(third_pred+second_pred)/2,
                  'price123':(first_pred+second_pred+third_pred)/3},index=id_test).head(10)